In [ ]:
import sys
import numpy as np
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers
import matplotlib.pylab as plt
import pickle
from pathlib import Path
from skimage.transform import resize

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
classes_names = ["2_clubs","2_diamonds","2_hearts","2_spades",\
               "3_clubs","3_diamonds","3_hearts","3_spades",\
               "4_clubs","4_diamonds","4_hearts","4_spades",\
               "5_clubs","5_diamonds","5_hearts","5_spades",\
               "6_clubs","6_diamonds","6_hearts","6_spades",\
               "7_clubs","7_diamonds","7_hearts","7_spades",\
               "8_clubs","8_diamonds","8_hearts","8_spades",\
               "9_clubs","9_diamonds","9_hearts","9_spades",\
               "10_clubs","10_diamonds","10_hearts","10_spades",\
               "ace_clubs","ace_diamonds","ace_hearts","ace_spades",\
               "jack_clubs","jack_diamonds","jack_hearts","jack_spades",\
               "king_clubs","king_diamonds","king_hearts","king_spades",\
               "queen_clubs","queen_diamonds","queen_hearts","queen_spades"]

In [ ]:
gen_params = {"samplewise_center":True,\
              "samplewise_std_normalization":True}

In [ ]:
generator = ImageDataGenerator(**gen_params, validation_split=0.15)

In [ ]:
bs = 32 # batch size

In [ ]:
path = Path("dataset/")

In [ ]:
img_height = 224
img_width = 224

In [ ]:
train_generator = generator.flow_from_directory(
    directory = path,
    target_size=(img_height, img_width),
    batch_size=bs,
    class_mode="categorical",
    subset='training',
    interpolation="nearest",
    shuffle=True,
    seed=69,
    classes=classes_names) # set as training data

In [ ]:
validation_generator = generator.flow_from_directory(
    directory = path,
    target_size=(img_height, img_width),
    batch_size=bs,
    class_mode="categorical",
    subset='validation',
    interpolation="nearest",
    shuffle=True,
    seed=69,
    classes=classes_names) # set as validation data

In [ ]:
#Exploratory data analysis
Xbatch, Ybatch = train_generator.__getitem__(0)

plt.figure(figsize = (32,24))

labels = Ybatch

for ii in range(int(bs/4)):
    plt.subplot(4,4,ii+1)
    plt.imshow((Xbatch[ii]- Xbatch[ii].min())/(Xbatch.max() - Xbatch[ii].min()), cmap = "gray")
    label = int(labels[ii].argmax())
    plt.title("Label: {}".format(label),\
              fontsize=30)
plt.show()

In [ ]:
trainable_flag = True
include_top_flag = False
weigths_value = 'imagenet'

if trainable_flag:
    include_top_flag = True
    weigths_value = None
else:
    include_top_flag = False
    weigths_value = 'imagenet'

In [ ]:
base_model = tf.keras.applications.VGG16(
    weights=weigths_value,
    input_tensor=None,
    pooling=None,
    classifier_activation="softmax",
    input_shape=(img_height, img_width, 3),
    include_top=include_top_flag,
    classes=len(classes_names) )

base_model.trainable = trainable_flag
inputs = layers.Input(shape=(img_height,img_width,3))
outputs = base_model(inputs)
model = tf.keras.Model( inputs,  outputs)

In [ ]:
print("Model Summary:")
print(model.summary())

In [ ]:
def scheduler(epoch, lr):
    if epoch%25 == 0 and epoch!= 0:
        lr = lr/(1.5)
    return lr

filepath_name='model_{epoch:d}.h5'

monitor_it = tf.keras.callbacks.ModelCheckpoint(filepath = filepath_name , monitor='val_loss',\
                                             verbose=1,save_best_only=False,\
                                             save_weights_only=False,\
                                             mode='auto',\
                                             save_freq='epoch')

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 40, min_delta=0.01)

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose = 0)

In [ ]:
num_epochs = 20
init_lr = 0.5e-4

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = init_lr),\
              loss='categorical_crossentropy',\
              metrics=['accuracy'])

In [ ]:
history_it = model.fit(train_generator, epochs=num_epochs, verbose = 1, \
                       workers=8, validation_data = (validation_generator),\
                       callbacks= [monitor_it, early_stop, lr_schedule], shuffle=True, initial_epoch=0)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 7))

axes[0].plot(history_it.history["loss"], label = "Loss")
axes[0].plot(history_it.history["val_loss"], label = "Val. loss")

axes[0].legend(prop={'size': 16})

axes[1].plot(history_it.history["accuracy"], label = "Acc.")
axes[1].plot(history_it.history["val_accuracy"], label = "Val Acc.")

axes[1].legend(prop={'size': 16})

plt.show()